## [Improve Accuracy of OCR using Image Preprocessing](https://medium.com/cashify-engineering/improve-accuracy-of-ocr-using-image-preprocessing-8df29ec3a033)
- OCR output highly depends on the quality of input image.    
- This is why every OCR engine provides guidelines regarding the quality of input image and its size. 
- These guidelines help OCR engine to produce accurate results.    
- Here Image Preprocessing comes into play to improve the quality of input image so that the OCR engine gives you an accurate output.
---
- OCR의 결과는 입력 이미지의 Quality에 많은 의존성을 가진다
- 이것은 모든 OCR Engine이 입력 이미지의 Quality와 이미지의 사이즈에 대한 가이드라인을 제공하는 이유이다
- 이러한 가이드라인은 OCR Engine이 정확한 결과를 생산하는 것을 도울것이다
- 이미지 전처리는 입력이미지의 Quality를 향상 시켜서 OCR Engine이 너에게 정확한 결과를 주도록 할 것 이다

In [22]:
from PIL import Image
import cv2
import numpy as np
import os
import tempfile

# imutils : OpenCV가 제공하는 기능 중에 좀 복잡하고 사용성이 떨어지는 부분을 잘 보완해 주는 패키지
# https://enjoyimageprocessing.tistory.com/entry/imutils-package-%EA%B8%B0%EB%8A%A5
import imutils


### Scaling of image 
- Image Rescaling is important for image analysis. 
- Mostly OCR engine give an accurate output of the image which has 300 DPI. 
- DPI describes the resolution of the image or in other words, it denotes printed dots per inch.
---
- 이미지 Rescaling은 이미지 분석에 매우 중요하다
- 대부분의 OCR Engine은 이미지가 300 DPI일때 정확한 결과를 준다
- DPI는 이미지의 Resolution(해상도)를 나타내고, 다른 말로 인치당 표시되는 도트의 수를 나타낸다


In [37]:
# 이미지 경로를 입력받아 300dpi로 변환 후 확장자는 png로 저장한다, return 저장된 파일 이름
def set_image_dpi(file_path):
    im = Image.open(file_path)
    
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    
    # import tempfile : 임시 파일 생성
    # https://medium.com/@silmari/python-tempfile-%EC%9E%84%EC%8B%9C%ED%8C%8C%EC%9D%BC-%EB%B0%8F-%ED%8F%B4%EB%8D%94-%EB%A7%8C%EB%93%A4%EA%B8%B0-86ea533086ce
    # delete=True : 코드 실행이 완료되면 임시 파일 삭제, True로 하면 파일이 저장이 안되어 나중에 읽을수가 없다
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')  
    temp_filename = temp_file.name
    
    im_resized.save(temp_filename, dpi=(300, 300))
    
    return temp_filename

### Skew Correction
- A Skewed image is defined as a document image which is not straight. 
- Skewed images directly impact the line segmentation of OCR engine which reduces its accuracy. 
- We need to process the following steps to correct text skew.
---
- Skewed(왜곡된, 비스듬한, 기울어진) Image는 똑바르지 않은 문서 이미지로 정의된다 
- Skewed Image는 OCR Engine의 Line Segmentation에 바로 영향을 준다, 이것은 정확도를 줄인다
- 우리는 기울어진 텍스트를 올바르게 만들기 위해 아래와 같은 처리가 필요하다

#### 1. Detect the text block with skew in the image.
---
#### 1. 이미지에서 기울어진 Text Block 탐지하기

In [38]:
def skew_correction(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 10, 50)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
    screenCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break

    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
    return image, screenCnt

#### 2. Now calculate the angle of rotation.
#### 3. Rotating the image to correct the skew.
----
#### 2. 회전 각도 계산
#### 3. 비스듬한 이미지를 바르게 만들도록 이미지 회전

In [39]:
# https://medium.com/@brijesh.vicky/thanks-for-writing-i-am-sending-you-detail-of-above-two-variable-8134ed7f3020
process_height = 500
ratio = image.shape[0] / float(process_height)
# And orig is copy of original image
orig = image.copy()

pts = np.array(screenCnt.reshape(4, 2) * ratio)
warped = four_point_transform(orig, pts)

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

NameError: name 'screenCnt' is not defined

### Binarization
- Mostly, an OCR engine does binarization internally because they work on Black & White images. 
- The simplest way is to calculate a threshold value and convert all pixels to white with a value above threshold value and rest of pixels convert into the black. 
- I am using OpenCV with Simple Thresholding, Adaptive Thresholding, and Otsu’s Binarization.
---
- 대부분의 OCR Engine은 내부적으로 이진화를 수행한다 그것들이 흑백 이미지에서 작동하기 때문이다
- 가장 간단한 방법은 Threshold(임계값) 값을 계산하는 것이다, Threshold 값보다 큰 픽셀값은 모두 하얀색으로 바꾸고 나머지 픽셀은 검은색으로 바꾼다
- OpenCV의 [Simple Thresholding, Adaptive Thresholding, and Otsu’s Binarization] 를 사용한다

### Noise Removal or Denoise
- Noise is a random variation of color or brightness between pixels of an image. 
- Noise decrease the readability of text from an image. 
- There are two major types of noise - salt and pepper noise and Gaussian noise.
---
- Noise는 이미지의 픽셀들 사이에 있는 밝기나 색깔의 랜덤한 변화이다
- Noise는 이미지에 있는 Text의 가독성을 감소한다 
- 주요한 두가지 종류의 Noise가 있다, Salt and Pepper Noise 그리고 Gaussian Noise

In [40]:
def remove_noise_and_smooth(file_name):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

### Useful Links 
- There are plenty of tools and articles for image processing on the internet. 
- I am putting up some useful links which can help you to implement these process to improve your accuracy.
---
- 인터넷에는 이미지 처리를 위한 많은 도구와 기사가 있다
- 정확도를 향상시키기 위한 처리를 시도하는데 도움이 될 유용한 링크를 마련했다    
[Image processing OpenCV](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_table_of_contents_imgproc/py_table_of_contents_imgproc.html)    
[Skew correction using python](https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/)    
[Mobile document scanner](https://www.pyimagesearch.com/2014/09/01/build-kick-ass-mobile-document-scanner-just-5-minutes/)    

In [41]:
def show_img(image):
    cv2.imshow('image', image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [44]:
def main():
    #파일 경로 설정
    cur_path = os.getcwd()
    img_path = cur_path + '/images/receipt1.jpg'
    
    #set_image_dpi 함수 호출
    tmp_file_path = set_image_dpi(img_path)
    image = cv2.imread(tmp_file_path, cv2.IMREAD_COLOR)
    print(tmp_file_path)
    show_img(image)
    
    #skew_correction 함수 호출
    image = skew_correction(image)
    show_img(image)
    
    

In [45]:
main()

/var/folders/tg/_j81051n4vb81m9k1pfpf58w0000gn/T/tmp43dgd1ok.png
